# Prosjekt 2 VitBer
Gruppe 49: Martin Johnsrud, Hans Giil, Eirik Høydalsvik

In [ ]:
%%javascript
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});

### Dimesjonsløshet

Sånn Shcrödingerlingingen står nå, vår man veldig små tall i bergeningene. Dette kan føre til flyttallfeil. Vi innfører derfor de dimesjonsløse størrelsene $ q = x / \omega $, $ \epsilon = E / (\hbar^2 / 2m\omega^2) $ og $ \hat \nu = \hat V / (\hbar^2 / 2m\omega^2) $, hvor $\omega$ er den karikteristiske størrelsen bredden til brønnen. Hele modellen får dermed en lengde $L^* = L / \omega$. Siden

$$ \frac{\partial^2}{\partial q^2} \psi(q) = \Big( \frac{\partial x}{\partial q} \frac{\partial}{\partial x} \Big)^2 \psi(x)= \omega^2 \frac{\partial^2}{\partial x^2} \psi(x), $$

kan vi ved å multiplisere med $2m\omega^2 / \hbar^2 $ på begge sider av Schrödingerligningen skrive den om til

\begin{equation}
    \label{TUSL}
    \Big(\hat \nu - \frac{\partial^2}{\partial q^2} \Big) \psi = \epsilon \psi
\end{equation}

Dette gjør at de nye elementene i $ \mathbb{H} $ matrisen blir $ H_{j, j} = 2 / (\Delta q)^2 + \hat \nu$ og $ H_{j, j \pm 1} = - 1 / (\Delta q)^2$, hvor $\Delta q = L^* / (n + 1)$.

**Notasjon**

$L^* =$ `LStar`  
$\Delta q$ = `deltaq`  
$n$ = `n`   
$\nu(q) =$ `vKont(q)`  
$\vec \nu$ = `v`  
$\epsilon_k =$ `eps[k]`  
$\psi = $ `psi[j][k]`    
$\vec \psi_k =$ `psi[:, k]`  
$\Psi(x, 0) = $ `f(x)`  
$\vec \Psi(t) = $ `Psi[j](t)`  
$\mathbb{H} = $ `H[j][k]`  

Løser (1) numerisk, som beskrevet i teorien. Returnerer $\epsilon$ og $\psi$.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.linalg as la
import numpy as np
from numpy import pi

def solveTUSL(v, LStar):
    
    # Hjj are diagonal values, HOD are of the diagonal
    deltaq = LStar / (len(v) + 1)
    Hjj = 2 / deltaq**2 * np.ones(len(v)) + v
    HOD = -1 / deltaq**2 * np.ones(len(v) - 1)
    return la.eigh_tridiagonal(Hjj, HOD)

Lager en krystall med `Nw` brønner med dybde `v0` (anntas negativ). returnerer $\epsilon,\, \psi,\, \nu,\, n$  og $L$. `L` blir beregnet slik at brønnene får bredde `wL` meter.

In [ ]:
def makeChrystal(Nw, v0):
    n = 2000 # Oppløsning
    # Størrelsene her er kun riktige i forhold til hverandre, enhetene stemmer
    # ikke overens med noe utenfor denne funksjonen
    w = 10   # Bredde mellom brønner som anndel av Length
    b = 3    # Avstand mellom brønner
    B = 10   # Bredde på sidene, i forhold til w
    Length = 2 * B * w + Nw * w  # Lengde av hele modellen
    if Nw > 1:
        Length += (Nw - 1) * b

    v = np.zeros(n * B * w // Length)
    if Nw:
        v = np.concatenate((v, v0 * np.ones(n * w // Length)))    
    for i in range(Nw - 1):
        v = np.concatenate((v, np.zeros(n * b // Length), v0 * np.ones(n * w // Length)))
    v = np.concatenate((v, np.zeros(n * B * w // Length) // Length))

    n = len(v)   # Forsikrer at v og x har samme lengde
    LStar = Length / w
    q = np.linspace(0, LStar, n)
    eps, psi = solveTUSL(v, LStar)
    return eps, psi, v, q

Plotter potensialbrønnen, `l` energiegenverdier ($\epsilon_k$) og de tilhørende egenvektorene $\vec \psi_k$. Venstre y-akse angir verider for $\epsilon$, mens høyre gir verdier for potensialbrønnen $\nu$. 

In [ ]:
def plotEigenfunc(eps, psi, v, q, l,  epsRange=(0, 0), legend=True, showWave=True):
    
    font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 25}
    plt.rcParams['mathtext.fontset'] = 'dejavuserif'
    mpl.rc('font', **font)
    
    fig, ax1 = plt.subplots(1, 1, figsize=(20, 10))
    
    inf = 1e10  # Alle vet at uendelig er ca 10 milliarder
    v = np.concatenate(([inf], v, [inf]))
    q = np.concatenate(([0], q, [q[-1] * (1 + 1 / len(q))]))
    
    # Inneholder Line2D elementer som viser legend hvordan linja ser ut
    legendElement = [0] * (l + 1)
    
    ax1.set_xlabel("$q$", fontsize=50)
    ax1.set_ylabel("$\epsilon $", fontsize=50)
    labels =  [0] * (l + 1)
    labels[0] = "$ \\nu (q)$"
    legendElement[0], = ax1.plot(q, v, color="Black")
    ax1.set_ylabel("$\\nu$", fontsize=50)

    scalePsi = 0
    if showWave: scalePsi = (epsRange[1] - epsRange[0]) / (5 * np.max(psi))

    for i in range(l):
        legendElement[1 + i], = ax1.plot(q[1: -1], psi[:, i] * scalePsi + eps[i] * np.ones_like(q[1: -1]))
        ax1.plot(q, eps[i] * np.ones_like(q), "--",  color= legendElement[1 + i].get_color())
        labels[1 + i] = "$\epsilon_" + str(i + 1) + "$"
    
    ax1.set_ylim(epsRange)
    ax1.set_ylim(epsRange)

    if legend:
        ax1.legend(legendElement[:][:l + 1], labels[:l + 1], fontsize=20)
    
    plt.show()

Konverterer fra den enhetsløse variablen $\epsilon$ til $E$, og gir verdier i elektronvolt eller joule.

In [ ]:
def EJ(eps):
    return eps * (1.0545e-34)**2 / (9.11e-31 * 1e-10**2)

def EeV(eps):
    return eps * (1.0545e-34)**2 /  (1.6e-19 * 9.11e-31 * 1e-10**2)

## Oppgave 1: Partikkel i boks

In [ ]:
eps1, psi1, v1, q1 = makeChrystal(0, 0)

l1 = 3
print(eps1[0])
plotEigenfunc(eps1, psi1, v1, q1, l1, (-eps1[0], eps1[l1 - 1] * 1.4))

### Energiegenverdier

Den analytiske løsningen for energien til en partikkel i boks er 

$$ \epsilon_k = \frac{2m\omega^2}{\hbar^2} E_k = \Big(\frac{k \pi}{L^*}\Big)^2. $$

Under printes både numeriske og analytisk beregnet dimensjonsløs energi, og man kan tydlig se at de stemmer godt overens.

In [ ]:
def epsExact(k, LStar):
    return (k * np.pi / LStar)**2

precision = 6
tableWidth = 15

print("Energies:")
print("Numerisk:".ljust(tableWidth), "Analytisk:".ljust(tableWidth), "Relativ feil:".ljust(tableWidth))

for i in range(l1):
    print(("{:.{prec}e}").format(eps1[i], prec=precision).ljust(tableWidth),\
          ("{:.{prec}e}").format(epsExact(i + 1, q1[-1]), prec=precision).ljust(tableWidth),\
          ("{:.{prec}e}").format((epsExact(i + 1, q1[-1]) - eps1[i]) / epsExact(i + 1, q1[-1]), prec=precision).ljust(tableWidth))

### Ortonormalitet og fullstendig sett

Vi kan se her at opp til en feil av størrelsesorden $10^{-10}$ er vektorene $ \vec \psi_k $ ortonormale, altså er $\vec \psi_i \cdot \vec \psi_j = \delta_{i, j}$, eller $\psi_{j, k} \cdot \psi_{j, k}^T = \mathbb{I}$, identitetsmatrisen, som sjekkes i koden under. Dette viser også at $\psi_{j, k}$ er inverterbar, som betyr at den er en basis til $ \mathbb{R}^n $, og dermed også et fullstendig sett.

In [ ]:
np.set_printoptions(precision=2)

p = psi1 @ psi1.T
print(p)

# Sjekker om diagonalen er nær 1, og resten nær 0
tol = 10e-10
i = 0
while(i < len(p[0]) and p[i, i] - 1 < tol and np.sum(abs(p[i,:i:])) < tol):
    i += 1
    
print("\nFeil innenfor toleransen: ", i == len(p))

## Oppgave 2: Atomer

In [ ]:
v0 = -70
eps2, psi2, v2, q2 = makeChrystal(1, v0)

l2 = 5
plotEigenfunc(eps2, psi2, v2, q2, l2, (eps2[0] * 1.2, -eps2[0] * 0.5))

for i in range(l2):
    print(eps2[i], q2[-1])

## Oppgave 3: Molekyler

In [ ]:
eps3, psi3, v3, q3 = makeChrystal(2, v0)

l3 = 7
plotEigenfunc(eps3, psi3, v3, q3, l3, (eps3[0] * 1.2, -eps3[0]*0.5))

for i in range(l3):
    print(eps3[i])

## Oppgave 4: Krystaller

In [ ]:
k = 10
eps4, psi4, v4, q4 = makeChrystal(k, -70)

l4 = k * 3 + 1
plotEigenfunc(eps4, psi4, v4, q4, l4, (eps4[0] * 1.2, -eps4[0]*0.5), legend=False, showWave=False)

## Bonus: Tidsutvikling

In [ ]:
from matplotlib.animation import FuncAnimation as anim
from IPython.display import HTML
import time
plt.rcParams.update({'animation.html':'html5', 'savefig.dpi': 50})

def f(x):
    return np.exp(-(x - 10)**2 + 1j*(x - 10)*40)

eps, psi, v, q = makeChrystal(0, 0)
c = psi.T @ f(q)
Psi = lambda t: psi @ (c * np.exp(-1j*eps*t))

fig, ax = plt.subplots(1, figsize=(15, 15))
T =- time.time()
numFrames = 100
def animate(n):
    global T
    ax.cla()
    ax.set_ylim(0, 1)
    t = n * 0.008 + 100
    l = ax.plot(q, np.abs(Psi(t))**2)
    T = T + time.time() 
    if n == numFrames - 1:
        print(T)
        print(T / frames)
    T = T - time.time()
    return l

a = anim(fig, animate, frames=numFrames, interval=80)
plt.close(fig)
HTML(a.to_html5_video())